<a href="https://colab.research.google.com/github/a9e68ce5/Machine-Learning/blob/main/PM%202.5%20Prediction%20by%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv
import math
import os
os.getcwd()
import random
seed=9487
np.random.seed(seed)
!gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"
!gdown --id '155N6fzI7vAFzHAGdy6jkaWIksWH6Y1G2' --output "test.csv"
data = pd.read_csv("/content/train.csv")
def valid(x, y):
  # TODO: Try to filter out extreme values.
  #  ex: If PM2.5 > 100, then we don't use the data to train (return False), otherwise return True,
 #data.corr()['PM2.5'].sort_values()
 #先將PM2.5的欄位標準化之後若超過正負三倍標準差則剔除
 mean=data[14].mean()
 std=data[14].std()
 y=(data[14]-mean)/std
 for i in range(5774):
   if (y[i]<3 and y[i]>-3):
     return True
   else:
     return False
def parse2train(data, feats):
#創造2個list,x,y
  x = []
  y = []

  # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
  total_length = data.shape[1] - 8 #data的行數

  for i in range(total_length):
    x_tmp = data[feats, i:i+8] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc.
    y_tmp = data[-1, i+8] # last column of (i+8)th row: PM2.5

    # Filter out extreme values to train.
    if valid(x_tmp, y_tmp):
      x.append(x_tmp.reshape(-1,))#自動計算列數
      y.append(y_tmp)

  # x.shape: (n, 15, 8)
  # y.shape: (n, 1)
  x = np.array(x)
  y = np.array(y)

  return x,y
def minibatch(x, y, config):

    # Randomize the data in minibatch
    index = np.arange(x.shape[0])#x的列數
    np.random.shuffle(index)
    x = x[index]
    y = y[index]

    # Initialization
    batch_size = config.batch_size
    lr = config.lr #learning rate
    lam = config.lam
    epoch = config.epoch

    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)#建立以x的列數為大小且都是0.9的n*1陣列
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1) #建立以x的列數為大小且都是0.99的n*1陣列
    # Linear regression: only contains two parameters (w, b).
    #Implement 2-nd polynomial regression version for the report.
    #y=w_1X^2+w_2X+b
    w_1 = np.full(x[0].shape, 0.1).reshape(-1, 1) #建立以x的列數為大小且都是0.1的n*1陣列
    w_2 = np.full(x[0].shape, 0.1).reshape(-1, 1) #建立以x的列數為大小且都是0.1的n*1陣列
    bias = 0.1
    m_t_1 = np.full(x[0].shape, 0).reshape(-1, 1) #建立以x的列數為大小且都是0的n*1陣列
    m_t_2 = np.full(x[0].shape, 0).reshape(-1, 1) #建立以x的列數為大小且都是0的n*1陣列
    v_t_1 = np.full(x[0].shape, 0).reshape(-1, 1) #建立以x的列數為大小且都是0的n*1陣列
    v_t_2 = np.full(x[0].shape, 0).reshape(-1, 1) #建立以x的列數為大小且都是0的n*1陣列
    m_t_b = 0.0 #b的momentum
    v_t_b = 0.0 #v的momentum
    t = 0
    epsilon = 1e-8 #10^-8

    # Training loop
    for num in range(epoch):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)
            x_x=x_batch*x_batch
            # Prediction of linear regression
            pred = np.dot(x_x,w_1)+np.dot(x_batch,w_2) + bias #y=w_1X^2+w_2X+b
            # loss
            loss = y_batch - pred #y-(w_1X^2+w_2X+b)

            # Compute gradient

            g_t_1 = np.dot(x_x.transpose(),loss) * (-2) +  2 * lam * np.sum(w_1)
            g_t_2 = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w_2)
            g_t_b = loss.sum(axis=0) * (-2)
            m_t_1 = beta_1*m_t_1 + (1-beta_1)*g_t_1
            m_t_2 = beta_1*m_t_2 + (1-beta_1)*g_t_2
            v_t_1 = beta_2*v_t_1 + (1-beta_2)*np.multiply(g_t_1, g_t_1)
            v_t_2 = beta_2*v_t_2 + (1-beta_2)*np.multiply(g_t_2, g_t_2)
            m_cap_1 = m_t_1/(1-(beta_1**t))
            m_cap_2 = m_t_2/(1-(beta_1**t))
            v_cap_1 = v_t_1/(1-(beta_2**t))
            v_cap_2 = v_t_2/(1-(beta_2**t))
            m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
            v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b)
            m_cap_b = m_t_b/(1-(0.9**t))
            v_cap_b = v_t_b/(1-(0.99**t))


            # Update weight & bias
            w_1 -= ((lr*m_cap_1)/(np.sqrt(v_cap_1)+epsilon)).reshape(-1, 1)
            w_2 -= ((lr*m_cap_2)/(np.sqrt(v_cap_2)+epsilon)).reshape(-1, 1)
            bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)


    return w_1,w_2, bias
from argparse import Namespace

# TODO: Tune the config to boost your performance.
train_config = Namespace(
    batch_size = 200,
    lr = 0.00000348,
    lam = 0.1,
    epoch = 20000,
)
feats = [1]
data = data.values
train_data = np.transpose(np.array(np.float64(data)))
train_x,train_y = parse2train(train_data,feats)
w_1,w_2, bias = minibatch(train_x, train_y,train_config)
data = pd.read_csv('test.csv')
data = data.values
def parse2test(data, feats):
  x = []
  for i in range(90):
    x_tmp = data[feats,8*i: 8*i+8]
    x.append(x_tmp.reshape(-1,))

  # x.shape: (n, 15, 8)
  x = np.array(x)
  return x
print(w_1,w_2,bias)
test_data = np.transpose(np.array(np.float64(data)))
test_x = parse2test(test_data, feats)
with open('my_sol.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])

    print(test_x.shape)
    for i in range(int(test_x.shape[0])):
      # Prediction of linear regression
      x_x=test_x[i]**2
      prediction = (np.dot(np.reshape(w_1,-1),x_x)+np.dot(np.reshape(w_2,-1),test_x[i]) + bias)[0]
      writer.writerow([i, prediction] )

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy
To: /content/train.csv
100% 324k/324k [00:00<00:00, 109MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1FXJztppYG9Q4b_4NHvcmPsc4o5obceWC
To: /content/test.csv
100% 49.0k/49.0k [00:00<00:00, 58.9MB/s]
[[-0.01706186]
 [-0.01291094]
 [-0.01097127]
 [-0.01180289]
 [-0.00947127]
 [-0.00930115]
 [-0.01150716]
 [-0.01446155]] [[0.7132499 ]
 [0.49535203]
 [0.41084807]
 [0.42030372]
 [0.40105526]
 [0.39206728]
 [0.51342679]
 [0.79584999]] [1.88097059]
(90, 8)
